In [26]:
import os
import pandas as pd

merged = pd.read_csv("merged_fred_stock_financial_w_nav.csv")

In [28]:
# --- Tag market stress, shock, and recession ---
merged["market_stress_flag"] = 0
merged["market_shock_flag"] = 0
merged["nber_recession_flag"] = 0

# Define recession (NBER-style macro periods)
recession_periods = [
    ("2000-03-01", "2002-10-31"),
    ("2007-12-01", "2009-06-30"),
    ("2011-08-01", "2011-08-31"),
    ("2020-03-01", "2020-03-31"),
    ("2022-01-01", "2022-06-30"),
    ("2023-03-01", "2023-03-31"),
]

for start, end in recession_periods:
    mask = (merged['date'] >= start) & (merged['date'] <= end)
    merged.loc[mask, "market_stress_flag"] = 1
    merged.loc[mask, "nber_recession_flag"] = 1

# Define market shock by VIX > 40 from best available column
vix_columns = [col for col in merged.columns if "vix" in col.lower()]
vix_column = None
for vix in ["vix_index", "linchpin__vix_index", "adj close_^vix"]:
    if vix in merged.columns:
        vix_column = vix
        break

if vix_column:
    merged.loc[merged[vix_column] > 40, "market_shock_flag"] = 1
    print(f"Shock tag applied using {vix_column}")
else:
    print("No VIX column found for shock tagging.")

# Save to disk
merged.to_csv("final_merged_dataset.csv", index=False)
print("Final dataset saved to final_merged_dataset.csv")

Shock tag applied using vix_index
Final dataset saved to final_merged_dataset.csv
